# 번역가는 대화에도 능하다

## Project: 멋진 챗봇 만들기

#### 사용할 라이브러리 임포트

필요한 패키지를 설치하고 불러옵니다.

In [1]:
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/v0.6.0/scripts/mecab.sh)

mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.


In [2]:
!pip install --upgrade gensim==3.8.3

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
import re
import os
import io
import time
import random
from sklearn.model_selection import train_test_split
from konlpy.tag import Mecab
import gensim
from gensim.models import KeyedVectors
from gensim.models.keyedvectors import Word2VecKeyedVectors
from tqdm import tqdm

### 데이터 다운로드

준비하기 단계에서 심볼릭 링크를 생성했다면 아래 파일이 ChatbotData .csv라는 이름으로 저장되어 있을거예요. csv 파일을 읽는 데에는 pandas 라이브러리가 적합합니다. 읽어 온 데이터의 질문과 답변을 각각 questions, answers 변수에 나눠서 저장하세요!

In [4]:
file_path = os.getenv('HOME') +'/aiffel/songys/Chatbot_data/ChatbotData.csv'
data = pd.read_csv(file_path)

In [5]:
data

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [6]:
print('Q 열에서 중복을 배제한 유일한 샘플의 수 :', data['Q'].nunique())
print('A 열에서 중복을 배제한 유일한 샘플의 수 :', data['A'].nunique())

Q 열에서 중복을 배제한 유일한 샘플의 수 : 11662
A 열에서 중복을 배제한 유일한 샘플의 수 : 7779


In [7]:
data.drop_duplicates(subset = ['Q'], inplace=True)
data.drop_duplicates(subset = ['A'], inplace=True)
print('전체 샘플수 :', (len(data)))

전체 샘플수 : 7731


In [8]:
data['Q'].values

array(['12시 땡!', '1지망 학교 떨어졌어', '3박4일 놀러가고 싶다', ..., '훔쳐보는 것도 눈치 보임.',
       '흑기사 해주는 짝남.', '힘든 연애 좋은 연애라는게 무슨 차이일까?'], dtype=object)

#### 질문데이터와 답변데이터로 분할

In [9]:
que = data['Q'].values

In [10]:
ans = data['A'].values

### 전처리와 Mecab을 통한 토큰화를 진행합니다.

In [11]:
from konlpy.tag import Mecab
mecab = Mecab()

In [12]:
def preprocess_sentence(sentence):
    
    sentence = sentence.lower()
    sentence = re.sub(r"[^0-9a-zㄱ-ㅎ가-힣?.!,]+", " ", sentence)
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = mecab.morphs(sentence)
    
    return sentence

In [13]:
def build_corpus():
    
    que_corpus, ans_corpus = [], []
    
    for i in range(len(que)):
        if 2 <= len(que[i]) <= 40 and 2 <= len(ans[i]) <= 40:
            que_corpus.append(preprocess_sentence(que[i]))
            ans_corpus.append(preprocess_sentence(ans[i]))
    
    return que_corpus, ans_corpus

In [14]:
que_corpus, ans_corpus = build_corpus()

In [15]:
len(que_corpus)

7649

In [16]:
len(ans_corpus)

7649

### Augmentation

In [18]:
ko_path = os.getenv('HOME') + '/aiffel/songys/Chatbot_data/ko.bin'

wv = gensim.models.Word2Vec.load(ko_path)

In [19]:
def lexical_sub(sentence, word2vec):
    import random
    
    res = ""
    toks = sentence

    try:
        _from = random.choice(toks)
        _to = word2vec.most_similar(_from)[0][0]
        
    except:
        return None

    for tok in toks:
        if tok is _from: res += _to + " "
        else: res += tok + " "

    return res

In [20]:
que_corpus_aug = que_corpus
ans_corpus_aug = ans_corpus
AUG_TIMES = 3

for mul in range(AUG_TIMES - 1):
    for i in tqdm(range(len(que_corpus))):
        que_corpus_new = lexical_sub(que_corpus[i], wv)
        ans_corpus_new = lexical_sub(ans_corpus[i], wv)

        if que_corpus_new is not None and ans_corpus_new is not None:
            que_corpus_aug.append(preprocess_sentence(que_corpus_new))
            ans_corpus_aug.append(preprocess_sentence(ans_corpus_new))

  0%|          | 0/7649 [00:00<?, ?it/s]/tmp/ipykernel_415/1927096422.py:9: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  _to = word2vec.most_similar(_from)[0][0]
100%|██████████| 13449/13449 [00:48<00:00, 274.94it/s]


In [21]:
len(que_corpus_aug)
len(ans_corpus_aug)

23875

### 데이터 벡터화

In [22]:
len(max(que_corpus_aug, key=len))

24

In [23]:
len(max(ans_corpus_aug, key=len))

24

In [24]:
for i in range(len(ans_corpus_aug)):
    ans_corpus_aug[i] = ["<start>"] + ans_corpus_aug[i] + ["<end>"]

In [25]:
MAX_LENGTH = 27

def tokenize(inputs, outputs):
    tokenized_inputs, tokenized_outputs = [], []
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=20000, filters=' ',oov_token="<unk>")
    tokenizer.fit_on_texts(inputs + outputs)
    
    tokenized_inputs = tokenizer.texts_to_sequences(inputs)   
    tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(tokenized_inputs, padding='post', maxlen=MAX_LENGTH)      
    
    tokenized_outputs = tokenizer.texts_to_sequences(outputs)   
    tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(tokenized_outputs, padding='post', maxlen=MAX_LENGTH)      
    
    
    return tokenized_inputs, tokenized_outputs, tokenizer

In [26]:
enc_train, dec_train, tokenizer = tokenize(que_corpus_aug, ans_corpus_aug)

In [27]:
tokenizer

In [28]:
enc_train.max()

7146

In [29]:
dec_train.max()

7512

In [30]:
START_TOKEN = [tokenizer.word_index["<start>"]]
END_TOKEN = [tokenizer.word_index["<end>"]]

In [31]:
print(START_TOKEN)
print(END_TOKEN)

[3]
[4]


In [32]:
VOCAB_SIZE = 7528

### 훈련하기

앞서 번역 모델을 훈련하며 정의한 Transformer 를 그대로 사용하시면 됩니다! 

대신 데이터의 크기가 작으니 하이퍼파라미터를 튜닝해야 과적합을 피할 수 있습니다.

In [33]:
def positional_encoding(pos, d_model):
    def cal_angle(position, i):
        return position / np.power(10000, int(i) / d_model)

    def get_posi_angle_vec(position):
        return [cal_angle(position, i) for i in range(d_model)]

    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(pos)])
    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])
    return sinusoid_table

In [34]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        
        self.num_heads = num_heads
        self.d_model = d_model
        self.depth = d_model // self.num_heads
        self.W_q = tf.keras.layers.Dense(d_model)
        self.W_k = tf.keras.layers.Dense(d_model)
        self.W_v = tf.keras.layers.Dense(d_model)
        self.linear = tf.keras.layers.Dense(d_model)

        
    def scaled_dot_product_attention(self, Q, K, V, mask):
        
        d_k = tf.cast(K.shape[-1], tf.float32)       
        QK = tf.matmul(Q,K,transpose_b=True)
        scaled_qk = QK/tf.math.sqrt(d_k)

        if mask is not None: scaled_qk += (mask * -1e9) 
        
        attentions = tf.nn.softmax(scaled_qk, axis=-1)
        out = tf.matmul(attentions, V)

        return out, attentions
        

    def split_heads(self, x):
        
        batch_size = x.shape[0]
        split_x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        split_x = tf.transpose(split_x, perm=[0, 2, 1, 3])

        return split_x

    def combine_heads(self, x):
        
        batch_size = x.shape[0]
        combined_x = tf.transpose(x, perm=[0, 2, 1, 3])
        combined_x = tf.reshape(combined_x, (batch_size, -1, self.d_model))

        return combined_x
    

    def call(self, Q, K, V, mask):
        
        WQ = self.W_q(Q)
        WK = self.W_k(K)
        WV = self.W_v(V)
        
        WQ_splits = self.split_heads(WQ)
        WK_splits = self.split_heads(WK)
        WV_splits = self.split_heads(WV)
        
        out, attention_weights = self.scaled_dot_product_attention(WQ_splits, WK_splits, WV_splits, mask)
        
        out = self.combine_heads(out)
        out = self.linear(out)

        return out, attention_weights

In [35]:
class PoswiseFeedForwardNet(tf.keras.layers.Layer):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        
        self.w_1 = tf.keras.layers.Dense(d_ff, activation='relu')
        self.w_2 = tf.keras.layers.Dense(d_model)

    def call(self, x):
        out = self.w_1(x)
        out = self.w_2(out)
            
        return out

In [36]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()

        self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)
        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask):

        residual = x
        out = self.norm_1(x)
        out, enc_attn = self.enc_self_attn(out, out, out, mask)
        out = self.dropout(out)
        out += residual

        residual = out
        out = self.norm_2(out)
        out = self.ffn(out)
        out = self.dropout(out)
        out += residual
        
        return out, enc_attn

In [37]:
class Encoder(tf.keras.Model):
    def __init__(self, n_layers, d_model, n_heads, d_ff, dropout):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.enc_layers = [EncoderLayer(d_model, n_heads, d_ff, dropout) 
                        for _ in range(n_layers)]
        
    def call(self, x, mask):
        out = x
    
        enc_attns = list()
        for i in range(self.n_layers):
            out, enc_attn = self.enc_layers[i](out, mask)
            enc_attns.append(enc_attn)
        
        return out, enc_attns

In [38]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()

        self.dec_self_attn = MultiHeadAttention(d_model, num_heads)
        self.enc_dec_attn = MultiHeadAttention(d_model, num_heads)

        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout = tf.keras.layers.Dropout(dropout)
    
    def call(self, x, enc_out, causality_mask, padding_mask):

        residual = x
        out = self.norm_1(x)
        out, dec_attn = self.dec_self_attn(out, out, out, padding_mask)
        out = self.dropout(out)
        out += residual

        residual = out
        out = self.norm_2(out)
        out, dec_enc_attn = self.enc_dec_attn(out, enc_out, enc_out, causality_mask)
        out = self.dropout(out)
        out += residual
        
        residual = out
        out = self.norm_3(out)
        out = self.ffn(out)
        out = self.dropout(out)
        out += residual

        return out, dec_attn, dec_enc_attn

In [39]:
class Decoder(tf.keras.Model):
    def __init__(self,
                 n_layers,
                 d_model,
                 n_heads,
                 d_ff,
                 dropout):
        super(Decoder, self).__init__()
        self.n_layers = n_layers
        self.dec_layers = [DecoderLayer(d_model, n_heads, d_ff, dropout) 
                            for _ in range(n_layers)]
                            
                            
    def call(self, x, enc_out, causality_mask, padding_mask):
        out = x
    
        dec_attns = list()
        dec_enc_attns = list()
        for i in range(self.n_layers):
            out, dec_attn, dec_enc_attn = \
            self.dec_layers[i](out, enc_out, causality_mask, padding_mask)

            dec_attns.append(dec_attn)
            dec_enc_attns.append(dec_enc_attn)

        return out, dec_attns, dec_enc_attns

In [40]:
class Transformer(tf.keras.Model):
    def __init__(self, n_layers, d_model, n_heads, d_ff, src_vocab_size, tgt_vocab_size, pos_len, dropout=0.2, shared=True):
        super(Transformer, self).__init__()
        
        self.d_model = tf.cast(d_model, tf.float32)
        self.enc_emb = tf.keras.layers.Embedding(src_vocab_size, d_model)
        self.dec_emb = tf.keras.layers.Embedding(tgt_vocab_size, d_model)
        self.pos_encoding = positional_encoding(pos_len, d_model)
        self.dropout = tf.keras.layers.Dropout(dropout)
        self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
        self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout)
        self.fc = tf.keras.layers.Dense(tgt_vocab_size)
        self.shared = shared

        if shared: self.fc.set_weights(tf.transpose(self.dec_emb.weights))

    def embedding(self, emb, x):
        
        seq_len = x.shape[1]
        out = emb(x)

        if self.shared: out *= tf.math.sqrt(self.d_model)

        out += self.pos_encoding[np.newaxis, ...][:, :seq_len, :]
        out = self.dropout(out)

        return out

        
    def call(self, enc_in, dec_in, enc_mask, causality_mask, dec_mask):
        
        enc_in = self.embedding(self.enc_emb, enc_in)
        dec_in = self.embedding(self.dec_emb, dec_in)
        enc_out, enc_attns = self.encoder(enc_in, enc_mask)
        dec_out, dec_attns, dec_enc_attns = \
        self.decoder(dec_in, enc_out, causality_mask, dec_mask)
        logits = self.fc(dec_out)
        
        return logits, enc_attns, dec_attns, dec_enc_attns

In [41]:
def generate_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

def generate_causality_mask(src_len, tgt_len):
    mask = 1 - np.cumsum(np.eye(src_len, tgt_len), 0)
    return tf.cast(mask, tf.float32)

def generate_masks(src, tgt):
    enc_mask = generate_padding_mask(src)
    dec_mask = generate_padding_mask(tgt)

    dec_enc_causality_mask = generate_causality_mask(tgt.shape[1], src.shape[1])
    dec_enc_mask = tf.maximum(enc_mask, dec_enc_causality_mask)

    dec_causality_mask = generate_causality_mask(tgt.shape[1], tgt.shape[1])
    dec_mask = tf.maximum(dec_mask, dec_causality_mask)

    return enc_mask, dec_enc_mask, dec_mask

In [42]:
class LearningRateScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(LearningRateScheduler, self).__init__()
        
        self.d_model = d_model
        self.warmup_steps = warmup_steps
    
    def __call__(self, step):
        arg1 = step ** -0.5
        arg2 = step * (self.warmup_steps ** -1.5)
        
        return (self.d_model ** -0.5) * tf.math.minimum(arg1, arg2)

In [43]:
learning_rate = LearningRateScheduler(512)

In [44]:
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

In [45]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

In [46]:
@tf.function()
def train_step(src, tgt, model, optimizer):
    gold = tgt[:, 1:]
        
    enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt)

    with tf.GradientTape() as tape:
        predictions, enc_attns, dec_attns, dec_enc_attns = model(src, tgt, enc_mask, dec_enc_mask, dec_mask)
        loss = loss_function(gold, predictions[:, :-1])

    gradients = tape.gradient(loss, model.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    return loss, enc_attns, dec_attns, dec_enc_attns

#### 데이터의 크기가 작아 과적합을 피하기 위해 하이퍼파리미터 튜닝해야 한다

In [47]:
transformer = Transformer(n_layers=2, d_model=512, n_heads=8, d_ff = 1024, src_vocab_size=20000, tgt_vocab_size=20000, pos_len=200, dropout=0.5, shared=True)

In [48]:
BATCH_SIZE = 64
EPOCHS = 20


for epoch in range(EPOCHS):
    total_loss = 0
    
    idx_list = list(range(0, enc_train.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm(idx_list)

    for (batch, idx) in enumerate(t):
        batch_loss, enc_attns, dec_attns, dec_enc_attns = \
        train_step(enc_train[idx:idx+BATCH_SIZE],
                    dec_train[idx:idx+BATCH_SIZE],
                    transformer,
                    optimizer)

        total_loss += batch_loss
        
        t.set_description_str('Epoch %2d' % (epoch + 1))
        t.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch + 1)))

Epoch 20: 100%|██████████| 374/374 [00:35<00:00, 10.62it/s, Loss 0.2723]


### 성능 측정하기

In [49]:
def evaluate(sentence, model, tokenizer):
    pieces = preprocess_sentence(sentence)
    tokens = tokenizer.texts_to_sequences([pieces])
    _input = tf.keras.preprocessing.sequence.pad_sequences(tokens, maxlen=enc_train.shape[-1], padding='post')
    ids = []
    output = tf.expand_dims(START_TOKEN, 0)

    for i in range(dec_train.shape[-1]):
        enc_padding_mask, combined_mask, dec_padding_mask = generate_masks(_input, output)
        
        predictions, enc_attns, dec_attns, dec_enc_attns = model(_input, output, enc_padding_mask, combined_mask, dec_padding_mask)
        
        predicted_id = tf.argmax(tf.math.softmax(predictions, axis=-1)[0, -1]).numpy().item()
        
        if END_TOKEN == [predicted_id]:
            result = tokenizer.sequences_to_texts([ids])
            result = "".join(result)
            return pieces, result, enc_attns, dec_attns, dec_enc_attns

        ids.append(predicted_id)
        output = tf.concat([output, tf.expand_dims([predicted_id], 0)], axis=-1)
    result = result
    return pieces, result, enc_attns, dec_attns, dec_enc_attns

In [50]:
def chatbot(sentence, model=transformer, tokenizer=tokenizer):
    pieces, result, enc_attns, dec_attns, dec_enc_attns = \
    evaluate(sentence, model, tokenizer)
    
    print('질문: %s' % (sentence))
    print('답변: {}'.format(result))

In [51]:
chatbot("제대로 하고 있는 걸까?")

질문: 제대로 하고 있는 걸까?
답변: 자신 이 생각 할 수 있 는 거 니 직접 적 인 생각 이 많 아요 .


In [52]:
chatbot("날씨가 좋네")

질문: 날씨가 좋네
답변: 하늘 보 고 웃 어 보 세요 .


In [53]:
chatbot("몇 살이야?")

질문: 몇 살이야?
답변: 당신 은 나 의 는 소중 한 일 이 많 지요 .


In [57]:
chatbot("어렵다 어려워")

질문: 어렵다 어려워
답변: 쉽 지 않 을 때 쯤 를 많이 어렵 게 해요 .


In [58]:
chatbot("점심 먹어도 배고프다")

질문: 점심 먹어도 배고프다
답변: 볼 거 예요 .


# 회고

앞에 노드랑 비슷하게 사용해서 비슷하게 하면 되는가 했는데 오류가 많이 걸려서 다른 방법을 사용해야 했음. 번역기라던지 챗봇을 만들고 있는데 아직 까지 만족할만한 챗봇은 아직 못 만들고 있는듯 다음노드에서는 좀더 괜찮은 결과물을 낼수 있을까?